CIVI Report location: https://plum.greenehillfood.coop/civicrm/report/instance/101?reset=1&output=criteria <br> Select Activity Details - Membership Related

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import datetime
import itertools
from datetimerange import DateTimeRange
import pickle
import json
from container_credentials import Credentials
import civiActivityReport_selectBestTrans as best

In [2]:
os.getcwd()


'/home/candela/pCloudDrive/pCloud Backup/mofongo-HP-EliteBook-840-G8-Notebook-PC/Documents/ghfc/membershipReportsCIVI/greeneHill'

In [3]:

#os.chdir('/home/candela/Documents/greeneHill/membershipReportsCIVI')
#os.chdir('/home/mofongo/Documents/ghfc/membershipReportsCIVI/membershipReportingLogicSampleReports')
os.chdir('/home/candela/pCloudDrive/pCloud Backup/mofongo-HP-EliteBook-840-G8-Notebook-PC/Documents/ghfc/membershipReportsCIVI/membershipReportingLogicSampleReports')
#/home/candela/Documents/greeneHill/membershipReportsCIVI/membershipReportingLogicSampleReports

Key 'activity reports' to import: 'selectActivityReport_20231112.csv'

In [4]:
activityReport = pd.read_csv('./civiSelectActivityReport_20251206.csv')

In [5]:
activityReport.columns = [i.replace(' ','_')+'_act' for i in list(activityReport.columns)]

In [6]:
#NOTE: Activity_Date_act field DOES NOT provide seconds
activityReport = activityReport.assign(Activity_Date_DT_act = pd.to_datetime(activityReport['Activity_Date_act'], format = '%Y-%m-%d %H:%M'))

Placeholder for the newly introduced "best" transaction selection module

In [7]:
activityReport = best.remove_controller(activityReport)

In [8]:
#email selection logic: if Source Email is an admin, then select Target Email
#alternatives: where Target Email is blank, select Source Email
activityReport = activityReport.assign(email_grouping = activityReport.apply(lambda x: x['Target_Email_act'] if pd.notnull(x['Target_Email_act']) else x['Source_Email_act'], axis = 1))

In [9]:
class Member_transactions(object):
    # a utility to process Member dictionaries: both status and type
    #I need both the prevailing membership type as of a particular date and the status/trigger that led to it; 
    #it might be nice to place a range of beg and end dates for a membership type
    #set the class attributes
    #perhaps can employ a class inheritence to use this blueprint for status lookups
    Subject_act = {'Trial Membership - ':'2 mo trial','Trial Membership - Status: New':'2 mo trial','Zucchini Plan -':'6 mo trial','Zucchini Plan - Status: New':'6 mo trial','to Zucchini Plan':'6 mo trial','Lettuce Plan -':'lettuce', 'Avocado Plan -':'avocado','to Avocado Plan':'avocado','Park Slope Member -':'park slope','Park Slope Member - Status: New':'park slope','Apple Plan -':'apple','to Apple Plan':'apple', 'Bushwick Member -':'bushwick','Bushwick Member - Status: New':'bushwick','Carrot Plan - ':'carrot', 'Household Plan - ':'household', 'to Lettuce Plan':'lettuce','to Carrot Plan':'carrot','to Bushwick Member':'bushwick','Central Brooklyn Food Co-op Member':'central brooklyn'}

# unique values of status_lookup
#'cancelled', 'care giving leave', 'deactivated', 'disability leave', 'general leave', 'medical leave', 'parental leave', #'technical activation', 'technical reactivation', 'trial conversion', 'trial expiration', 'winback', 'suspended
    status_lookup = {'Active to General Leave':'general leave', 'to Cancelled':'cancelled', 'General Leave to New':'winback',  
                    'New to Expired':'trial expiration', 'to Deactivated':'deactivated', 'Deactivated to General Leave':'general leave','Deactivated to Expired':'trial expiration', 'to Expired':'trial expiration','Cancelled to Active':'winback','Active to Reactivated':'technical reactivation','to Parental Leave':'parental leave', 'General Leave to Active':'winback', 'Cancelled to Re-activated':'winback', 'to General Leave': 'general leave', 'Deactivated to Re-activated':'winback', 'Deactivated to Active':'winback','to Medical Leave':'medical leave', 'General Leave to Re-activated':'winback','Medical Leave to Re-activated':'winback', 'Expired to Active':'trial conversion', 'Parental Leave to Active':'winback','Expired to New':'trial conversion','Active to Re-activated':'technical reactivation','Disability Leave to Active':'winback','Active to Care Giving Leave':'care giving leave','Active to New':'technical activation','Expired to Re-activated':'trial conversion','New to Re-activated':'technical reactivation','Medical Leave to Active':'winback','Care Giving Leave to Active':'winback','Re-activated to New':'technical activation', 'Parental Leave to Re-activated':'winback','Active to Disability':'disability leave','Deactivated to New':'technical reactivation','Active to Suspended':'suspended','Suspended to Active':'winback','Suspended to Re-activated':'winback'}
    #second/higher level lookup: to be conducted after lower level lookups; not currently (3/16/25) in use
    member_abstract={'2 mo trial':'trial','lettuce':'full member','6 mo trial':'trial','avocado':'full member',
                     'carrot':'full member','bushwick':'full member','park slope':'full member','apple':'full member','central brooklyn':'full member'}
    #NOTE dict_loop() controls which dictionary is referenced; a/o 3/16/25 member_abstract IS NOT referenced
    lookup_dict_finder = {'type':Subject_act, 'status':status_lookup,'type_abstract':member_abstract}                     
    ''' candidate 'winback' status changes: 
    Cancelled to Re-activated
    Deactivated to Re-activated
    General Leave to Re-activated
    Medical Leave to Re-activated
    '''
    def __init__(self, dicty):
        #accept the transactional dictionary (by member)
        self.dicty = dicty
        self.class_dict = None #placeholder (this will be revised, if need-be, in dict_loop elif statements)
        return None
    
    #returns the key, which I use to look up the value in the lookup dict
    #designed to be multi-faceted (accepts any lookup dict)
    def dictCrawl(self,substring,lookup_dict):
        result_list = [i for i in lookup_dict.keys() if i in substring] 
        if len(result_list) == 0:
            return None
        else:
            #return the value from the lookup dict, after honing in on the matching key to the substring
            return lookup_dict[result_list[0]]
    
    #loop through ea key of dicty, pass through a string search function and return the corresponding dict value 
#from the initial vetting dict into the new results dict
#believe that the below also appends a seq number to the value retrieved from the dict
# have to swap the key of 
    def dict_loop(self,lookup_dict_key):
        
        def apply_lookup(base_dict, lookup_key):
            #base_dict will contain the raw values from activityReport
            new_results = {} # will store the keys from the lookuptable and the values from the member dict
            seq_num = 0
            for k,v in base_dict.items(): # dicty = member dictionary
                new_key = self.dictCrawl(k,self.lookup_dict_finder[lookup_key])
                if new_key is None:
                #do something
                    print(f"new member type doesn't match any lookup keys: {k}")
                else:
                #write results to a new dict of format: {looked-up key: [original Subject_act,Date_DT_act]} <- this can be readily fed into the DF that is later imported to the db
                    new_results[new_key+'_'+str(seq_num)] = [k,v]
                    seq_num= seq_num+1
            return new_results
        
        # if/then statements serves as a map of lookup_dict_key to "base" dict
        if lookup_dict_key == 'type_abstract':
            try: 
                #feed the class_dict (set at the elif statements below) into the apply lookup function to convert to abstract dict; 
                self.type_abstract_dict = apply_lookup(self.class_dict,'type_abstract')
            except NameError: #check if class_dict has already been assigned
                raise('the type dict does not yet exist; run dict_loop with lookup key = class')
        elif lookup_dict_key == 'type':
            self.class_dict = apply_lookup(self.dicty,'type')    
            return self.class_dict
        elif lookup_dict_key == 'status': # status should be handled at the class level so that all attributes are attached to same object
            self.class_dict = apply_lookup(self.dicty,'status')
            return self.class_dict
        else: # OK to proceed
            return None
    

In [10]:
class Member(object):
    #cobble together attributes of a particular member by extracting activity from the df
    
    #class attribute, bc never changes; the activity_df object would need to exist before this assignment
    #membership_df = activity_df

    #ideally I store the following attributes of a member: Ea status row carries with it: 1) applicable "type", 2) prevailing "type", 3) "status" lookup value (from Membership_transactions)
    
    def __init__(self, activity_df, email):
        self.email = email
        #instance attribute, but perhaps can be moved to a class attribute
        self.membership_df = activity_df
        
        #can choose to call email_lookup() here
        self.email_lookup()

        self.convertType_toDict() #creates self.dicty_type
        self.convertStatus_toDict() #creates self.dicty_status
        

    def email_lookup(self):
        #search the activity df for all instances of the email: both membership type changes and status changes
        try:
            # membership type change
            self.df = self.membership_df.loc[self.membership_df['email_grouping'].str.strip() ==self.email,:].sort_values(by = 'Activity_Date_DT_act')
            #remove some trivial activities
            self.df2 = self.df.loc[~self.df['Subject_act'].isin(['Status changed from New to Active','Status changed from Pending to New','Status changed from Re-activated to Active']),]
            #write a sequential key to a series on the df: this is used to account for activity (Status or Type) that repeats (ex. multiple trials, etc), because when written to a dict below (convertType_toDict and convertStatus_toDict), the keys must be unique
            self.df2 = self.df2.assign(seq_num = range(0,len(self.df2.index)))
            #convert to str 
            self.df2 = self.df2.assign(seq_num = self.df2['seq_num'].astype(str))
            #concat two Series
            self.df2 = self.df2.assign(Subject_act = self.df2['Subject_act'].str.cat(self.df2['seq_num'], sep = '_'))

            if self.df2.size == 0:
                #print('no match for email found')
                pass
            else:
                #print('email found')
                pass
        except Exception as e:
            # return something if the member email is not found in the activty dataframe
            print(e)
            
    #consider creating a separate class for all the data manipulation and lookup conversions
    def convertType_toDict(self):
        #filter on 'Activity_Type_act': {'Change Membership Status','Membership Signup','Change Membership Type'}
        #convert two series from the member df to a dict for future mapping/conversion
        dicty = {}
        #select for Membership types
        self.type_dict = self.df2.loc[self.df2['Activity_Type_act'].isin(['Membership Signup','Change Membership Type']),['Subject_act','Activity_Date_DT_act']]
        for i in self.type_dict.values:
            dicty[i[0]] = i[1]
        self.dicty_type_member = dicty
        return self.dicty_type_member
    
    def convertStatus_toDict(self):
        #filter on 'Activity_Type_act': {'Change Membership Status','Membership Signup','Change Membership Type'}
        #convert two series from the member df to a dict for future mapping/conversion
        dicty = {}
        #select for Membership types
        for i in self.df2.loc[self.df2['Activity_Type_act'].isin(['Change Membership Status']),['Subject_act','Activity_Date_DT_act']].values:
            dicty[i[0]] = i[1]
        self.dicty_status = dicty
        return self.dicty_status
    
    def interact_member_transactions(self):
        #interact with the Member_transactions class returning normalized dictionaries for "type" and "status"
        try:
            #member_transactions is generated from the {subject_act:Date_act} dictionary processed in converStats_toDict and convertType_toDict above
            member_trans = Member_transactions(self.dicty_type_member)
            #dict_loop will return the post lookup dict (handled in Member_transactions), format: {looked-up subject: [original subject,activity date]}
            dict_type = member_trans.dict_loop('type')
            assert len(self.dicty_type_member) == len(dict_type), "lengths of self.type_dict and dict_status in class Member don't match"
        except Exception as e:
            print("error generated in class Member.interact_member_transactions on 'type' process")
            print(e)

        try:
            member_trans = Member_transactions(self.dicty_status)
            #dict_loop will return the post lookup dict (handled in Member_transactions); dict format: {looked-up subject: [original subject,activity date]}
            dict_status = member_trans.dict_loop('status')
        except Exception as e:
            print("error generated in class Member.interact_member_transactions on 'status' process")
            print(e)

        return dict_type,dict_status

    def normalize_member_class(self):
        #will lookup whether a member is full, trial, et al
        return None
    
    '''
    #print intermittent output of the Member classes in order to determine best step at which to apply the trial functions
aatiell_class = Member(activityReport,'aatiell@gmail.com')
mt,ms = aatiell_class.interact_member_transactions()
print(mt)
print(ms)

    '''

### Handling trial expirations and conversions
Contemplating applying a ruleset: if the trial-related activity (cancellation, conversion, etc) occurs within the trial window (can be 1,2,6 months), then the 'type' activity is assigned to that trial
If there is no subsequent 'type' initiation/activation after the latest trial, then assign the 'status' to the last trial -> this will cover activity outside of trial date range

- Consider creating a trial rollover whereby a member rolls over from a 1 to 6 month trial

CASES: 
- 405sarah@gmail.com enrolls in a two month trial, which I need to relabel at the Member level
- 009janaki@gmail.com rolls a 1 month trial over to a 6 month trial INSIDE the window of the first 1 mo period
- aaikolord@gmail.com converts a trial to a full membership after trial expiration - implement a time cap for conversion
- aatiell@gmail.com: accurately encoded the status type to trial conversion
- abrennan@wesleyan.edu: re-code for 2 month trial expiration
- adam.b.neese@gmail.com: case study of a legit 1 mo trial
- adwait.spotify@gmail.com: may have simultaneously enrolled in two trials (one and two month)
- aef@fastmail.com: rolled from 1 month to 6 month
- aelewis22@yahoo.com: two trials and two conversions to full
- aidan.l.ling@gmail.com: conversion after trial expiration
- akari@tribecapediatrics.com: logged two identical trials on the same date (effectively one should be ignored)
- alembeck5@gmail.com: conversion to full within trial window and no expiration recorded for the corresponding trial
- alina@alinarancier.com: convert after trial expiration, and later go on to have many winbacks

### Regarding trials
Easiest route may be to associate all activity to trials having a start date BEFORE the date of activity; this may handle cases of multiple trials and actions in an effort to avoiding associating activity to the wrong trial

In [11]:
class Trial_pattern(object):
    #LIMITATION: if a trial was started prior to the date extract out of CIVI and the member converted AFTER the start date of the extract, the member will be erroneously have their trial flag set to NULL; I'll need to address this via an SQL stored procedure
    # TODO add a trial flag to each email grouping with a Trial indicator and the start_dt of their last trial
    #purpose of this class is to create the infrastructure to properly evaluate trial members: relate status records to the appropriate membership types, including both expirations and conversions
    #non-trials are more straightforward because the status change will correspond to the prevailing "type" on activity date
    #this version of Trial_pattern class differs from the above in that it is suited to dataframe INPUT as opposed to the dictionaries from the intermittent output of the Member class
    #designed to ingest the entire Member Type df at once, and set a "theoretical" expiration for ea trial based on Travis' critera:
    '''
    Travis: I can't find the exact date trials were changed from 30 days to 60, but it was around December 20th 2021.
    
    Me: So to be clear, are you saying that all trials (not Zucchini) prior to December '21 should have an expiration of one month and those initiated after that date should have a length of 2 months?
     
    '''
    threshold = datetime.datetime.strptime('2021-12-20', '%Y-%m-%d')

    def __init__(self, df):
        #pull both the 'type' and 'status' dictionaries from a member
        self.df_orig = df.copy()
        #this will change the original df object as well
        self.df_orig['trial_expiration'] = np.nan
        #create df of all trials by filtering for all records having 'trial' in the 'type' description
        self.df_trial = df.loc[df['type'].str.contains('trial'),:]

        #self.process()
        #class variable switch date for trials
        

    def loop_thru(self):
        # for ea record, select the expiration date by date of trial enrollment; per Travis 12/20/21 is the switch-over date from 1 mo to 2 mo trial lengths

        def define_expirations(df_row):
        # associate ea expiration entry in status changes to the appropriate trial (can be multiple trials for a member)
        # the value of this dict can be added directly to a datetime object, ex. datetime.datetime(2020,3,7) + value
        # a typical matching record will have {'trial type': datetime.date(2019, 12, 5)}
            time_deltas = {'1 mo trial':datetime.timedelta(days = 30), '2 mo trial':datetime.timedelta(days = 60), '6 mo trial': datetime.timedelta(days = 180)}
        
            def assign_expiry(df_row):
                if '6 mo trial' in df_row['type'] :
                    trial_expiration = '6 mo trial'
                elif df_row['start_dt'] < Trial_pattern.threshold:
                    trial_expiration = '1 mo trial'
                elif df_row['start_dt'] >= Trial_pattern.threshold:
                    trial_expiration = '2 mo trial'
                else:
                    None
                return trial_expiration
            #define_expirations returns below value
            return df_row['start_dt']+time_deltas[assign_expiry(df_row)]
        #change the class variable
        self.df_trial = self.df_trial.assign(trial_expiration = self.df_trial.apply(define_expirations, axis = 1))

        return None
        
    def process(self):
        #call function to alter self.df_trial
        self.loop_thru()

        self.df_non_trial = self.df_orig.loc[~self.df_orig['type'].str.contains('trial'),:]
        #concatenate both dataframes
        #FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

        return pd.concat([self.df_trial,self.df_non_trial])
    
    #build the trial flag: has to take in the entire df whole, then conduct all grouping, etc
    def append_last_trial(self,df_1):
    #isolate the trials and perform idxmax
    # if there are no trials, will need to catch and isolate
    # in order to test, can run it on email = jodyq@me.com', 
        df_2 = df_1.copy()
        df_2.reset_index(inplace=True)
        df_len = len(df_1)
    #print(f'length of df {df_len}')
        try:
            seg = df_2.loc[df_2['type_clean'].str.contains('trial'),:]
            #handle two cases: whether a trial exists in the grouped df or not
            if seg.size == 0:
            #return an empty array: be sure that data types are equal (np.NAN)
                temp_list = [{'start_dt': None,'type_clean':None}] * df_len
                df_2['latest_trial'] = pd.Series(temp_list)
                #print('returning seg size = 0 version')
                return(df_2)
            else:
            #last trial is a series; idxmax() returns the index # for the row with the latest start_dt
                last_trial = seg.loc[seg['start_dt'].idxmax(),['start_dt','type_clean']]
            #TODO Is this causing me to lose the time component? convert the datetime object start_dt to isoformat for easier json serializing later 
                last_trial['start_dt'] = last_trial['start_dt'].isoformat()
                hold = last_trial.to_dict()
            #bc pandas doesn't have vectorization, I need to create my own: duplicating the list for as many rows in the df
                temp_list = [hold] * df_len
            #latest_trial Series is now a Series of lists of size two
                sing = pd.DataFrame(pd.Series(temp_list))
                sing.columns = ['latest_trial']
                #print('returning seg size > 0 version')
                df_3 = pd.concat([df_2,sing], axis = 1)
                return(df_3)
        except ValueError as e:
            print(e)

    def process_append_last_trial(self, df_1):
        #merely interacts with append_last_trial function
        #DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

        processed_df = df_1.groupby('email').apply(self.append_last_trial)
        return(processed_df.reset_index(drop=True))
            

In [40]:

# STRICTLY FOR TESTING... IGNORE IGNORE
def append_last_trial(df_1):
    #isolate the trials and perform idxmax
    # if there are no trials, will need to catch and isolate
    df_2 = df_1.copy()
    df_2.reset_index(inplace=True, drop = True)
    df_len = len(df_1)
    #print(f'length of df {df_len}')
    try:
        seg = df_2.loc[df_2['type_clean'].str.contains('trial'),:]
        if seg.size == 0:
            #return an empty array: be sure that data types are equal (np.NAN)
            temp_list = [{'start_dt': None,'type_clean':None}] * df_len
            df_2['latest_trial'] = pd.Series(temp_list)
            df_2.reset_index(drop = True, inplace = True)
            #print('returning seg size = 0 version')
            return(df_2)
            
        else:
            #last trial is a series; idxmax() returns the index # for the row with the latest start_dt
            last_trial = seg.loc[seg['start_dt'].idxmax(),['start_dt','type_clean']]
            hold = last_trial.to_dict()
            #bc pandas doesn't have vectorization, I need to create my own: duplicating the list for as many rows in the df
            temp_list = [hold] * df_len
            #print(f'this it temp_list: {pd.Series(temp_list)}')
            #print(f'this is the df just before writing:{df_2}')
            #latest_trial Series is now a Series of lists of size two
            sing = pd.DataFrame(pd.Series(temp_list))
            sing.columns = ['latest_trial']
            #print(f'this is the df just after writing:{df_3}')
            #print('returning seg size > 0 version')
            df_3 = pd.concat([df_2,sing], axis = 1, ignore_index = True)
            df_3.reset_index(drop = True, inplace = True)
            return(df_3)
    except ValueError as e:
        print(e)

In [12]:
# compile by processing a dictionary of subject (post member_transactions lookup) and Activity_Date_DT_act and combine with columns for email and date range; NOTE: type field is present in both sets of DF (type & status)
# any desired changes to the final df (those inserted into the mysql db) can be done here in addition to the Member and Member_transactions classes
def create_df(mt_v):
    if len(mt_v) == 0:
        return None
    else:
    #will need to account for single row membership activity
        dfj = pd.DataFrame.from_dict(mt_v, orient = 'index').reset_index()
    #at this point can remove the sequencing from type_raw to write a cleaner "subject" to the db
        dfj.columns = ['type','type_raw','start_dt']
    #convert dtype
    #TODO ensure that to.datetime isn't dropping the time component
        dfj['start_dt'] = pd.to_datetime(dfj['start_dt'])
        dfj.sort_values('start_dt', ascending = True)
    #TODO: handle transactions where two or more made on same day... may actually want to delete all but the latest record
        dfj['lead_date'] = dfj['start_dt'].shift(-1)
        dfj['lead_date'] = dfj['lead_date'].apply(lambda x: x-datetime.timedelta(days = 1))
    #replace NaN with today's date
        djf = dfj.assign(lead_date = dfj['lead_date'].fillna(datetime.datetime.today().strftime(format = "%Y-%m-%d %H:%M:%S")))
    # if dataframe is of row length = 1, then the "shifted" column should = today's date
    #create a DateRange object per row
        djf['datetimerange'] = djf.apply(lambda x: DateTimeRange(x['start_dt'],x['lead_date']), axis = 1)
        # sanitize the type field by removing the sequence encoding and storing the abstracted type in its own field
        djf['type_clean'] = djf['type'].str.split(pat = '_', n = 1,expand = True)[0]

        return djf
    
#called independently from create_df()
def compile(unq_email):
    unq_mem_class = Member(activityReport,unq_email)
    mt,ms = unq_mem_class.interact_member_transactions() #returns two dictionaries: 1) contains "type" records; 2) contains "status" records; the dicts are of format: {converted subject line (via lookup): [original subject line, activity date]}
    return(mt,ms)

In [13]:
#collect both sets of activity (type and status) into their respective dataframes
# each DF will have the same columns: type (activity name), start_dt, lead_dt (date offset from proceeding activity or today's date), datetimerange, email
list_unq_email = activityReport.email_grouping.unique()
cum_df_type = pd.DataFrame()
cum_df_status = pd.DataFrame()
#iterate through ea email address and process: create a Member object
for i in list_unq_email:
    try:
        #generate both a type and status dict for ea email
        mt,ms = compile(i)
        mt_df = create_df(mt)
        ms_df = create_df(ms)
        if mt_df is None: # no type activity for member
            next
        else:
        #add member email to the df
            mt_df['email'] = i
            cum_df_type = pd.concat([cum_df_type,mt_df], axis = 0)
            
        if ms_df is None: # if no status activity for member, proceed
            next
        else:
        #add member email to the df
            ms_df['email'] = i
            cum_df_status = pd.concat([cum_df_status,ms_df], axis = 0)
        
    except Exception as e:
        print(f'error encountered with {i}; error message {e.args}')   

cum_df_type = cum_df_type.reset_index(drop = True)        

In [14]:
#apply the new trial expiration logic
df = Trial_pattern(cum_df_type)
#process() method of the Trial_pattern class adds a 'trial_expiration' field, which may be null in the cases of no trials; and that could trigger a Warning (concatenating to a NULL dataframe)
df2 = df.process()
df2 = df2.reset_index(drop = True)
#print(f'df2 second rnd: {df2.head()}')
type_df = df.process_append_last_trial(df2)
type_df.drop('index',inplace = True, axis = 1)
#print(f'status_df: {status_df.head()}')
#df.process().sort_values(['email','start_dt']).to_csv('./inspect.csv', index = False)

/tmp/ipykernel_30570/1165738459.py:62: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([self.df_trial,self.df_non_trial])
/tmp/ipykernel_30570/1165738459.py:103: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  processed_df = df_1.groupby('email').apply(self.append_last_trial)


In [15]:
print(df2.columns)
print(cum_df_type.columns)

Index(['type', 'type_raw', 'start_dt', 'lead_date', 'datetimerange',
       'type_clean', 'email', 'trial_expiration'],
      dtype='object')
Index(['type', 'type_raw', 'start_dt', 'lead_date', 'datetimerange',
       'type_clean', 'email'],
      dtype='object')


## QA: date range, df shapes

In [15]:
#QA: date range
print(type_df['start_dt'].min())
print(type_df['start_dt'].max())

2025-09-08 09:20:00
2025-12-05 14:44:00


In [16]:
print(cum_df_status['start_dt'].min())
print(cum_df_status['start_dt'].max())

2025-09-08 02:54:00
2025-12-06 01:54:00


In [15]:
#inspect lengths of each df
print(cum_df_status.shape)
print(type_df.shape)
print(activityReport.shape)
#reconcile the difference btwn sum of status and type df to length of activityReport
print(activityReport.loc[~activityReport['Subject_act'].isin(['Status changed from New to Active','Status changed from Pending to New','Status changed from Re-activated to Active']),].shape)

(1247, 7)
(1615, 9)
(3966, 11)
(2863, 11)


In [17]:
#QA the distribution of type_clean from either table
type_df['type_clean'].value_counts(dropna = False)

type_clean
2 mo trial          747
park slope          416
lettuce             234
6 mo trial          105
central brooklyn     50
carrot                2
Name: count, dtype: int64

In [26]:
#QA the distribution of type_clean from either table
cum_df_status['type_clean'].value_counts(dropna = False)

type_clean
trial expiration          208
winback                    27
deactivated                25
cancelled                  21
general leave              17
trial conversion           15
parental leave              3
technical reactivation      2
medical leave               2
care giving leave           1
Name: count, dtype: int64

At this point I have two tables that I'd like to import into the db: **type_df** and **cum_df_status**

In [17]:
type_df = type_df.assign(latest_trial2 = type_df['latest_trial'].apply(lambda x: json.dumps(x)))
type_df['ingest_date'] =  datetime.datetime.today() #datetime.datetime.strptime('2023-07-30', '%Y-%m-%d')
cum_df_status['ingest_date'] = datetime.datetime.today() #datetime.datetime.strptime('2023-07-30', '%Y-%m-%d')
#type_df.to_pickle('/home/candela/Documents/greeneHill/membershipReportsCIVI/membership_df2.p')

In [18]:
cum_df_status.head()

,type,type_raw,start_dt,lead_date,datetimerange,type_clean,email,ingest_date
0,trial expiration_0,Status changed from Active to Expired_0,2025-09-08 02:54:00,2025-12-06 10:11:07,2025-09-08T02:54:00 - 2025-12-06T10:11:07,trial expiration,alexis.m.gay@gmail.com,2025-12-06 10:11:48.733766
0,trial expiration_0,Status changed from Active to Expired_0,2025-09-08 02:54:00,2025-12-06 10:11:07,2025-09-08T02:54:00 - 2025-12-06T10:11:07,trial expiration,cmarcantonio@gmail.com,2025-12-06 10:11:48.733766
0,trial expiration_0,Status changed from Active to Expired_0,2025-09-08 02:54:00,2025-12-06 10:11:07,2025-09-08T02:54:00 - 2025-12-06T10:11:07,trial expiration,margenett@gmail.com,2025-12-06 10:11:48.733766
0,deactivated_0,Status changed from Active to Deactivated_0,2025-09-17 07:56:00,2025-10-25 15:59:00,2025-09-17T07:56:00 - 2025-10-25T15:59:00,deactivated,leahbass25@gmail.com,2025-12-06 10:11:48.733766
1,winback_1,Status changed from Deactivated to Active_1,2025-10-26 15:59:00,2025-12-06 10:11:07,2025-10-26T15:59:00 - 2025-12-06T10:11:07,winback,leahbass25@gmail.com,2025-12-06 10:11:48.733766


## Pickle interaction

In [ ]:
os.getcwd()

In [27]:
#using pickle to persist objects; can also use pandas to_pickle() method, which is more reliable
#filename1 = '/home/candela/Documents/greeneHill/membershipReportsCIVI/membership_df2.p'
#filename2 = '/home/candela/Documents/greeneHill/membershipReportsCIVI/membership_cum_df_status.p'
#filehandler = open(filename1, 'wb') 
#pickle.dump(df2, filehandler)
#filehandler = open(filename2, 'wb') 
#pickle.dump(cum_df_status, filehandler)

folder = '/home/candela/Documents/greeneHill/membershipReportsCIVI'
type_df.to_pickle(folder+'/membership_df2.p')
cum_df_status.to_pickle(folder+'/membership_cum_df_status.p')

In [15]:
type_df = pd.read_pickle('/home/candela/Documents/greeneHill/membershipReportsCIVI/membership_df2.p') #reads successfully
#cum_df_status doesn't exist
#cum_df_status = pd.read_pickle('/home/candela/Documents/greeneHill/membershipReportsCIVI/membership_cum_df_status.p')

### SQL import

In [19]:
import sqlalchemy
#from sqlalchemy import URL

In [27]:
engine.dispose()

In order to be able to use port forwarding, I have to set up an ssh connection to Radish on my terminal beforehand via an ssh command (NOTE I don't need to use Tailscale): <br>
>ssh -4 -L 127.0.0.1:5433:localhost:3306 lcalleja@67.207.80.236

In [28]:
#'candela':{'user':'root','pass':'salmon01','database':'membership','port':3306,'host':'172.17.0.2'}
#using ssh port forwarding, connecting my port 5433 and remote port 3306
user = 'lcalleja2'
password = '3059891242'
host = '127.0.0.1' #tailscale IP address 100.102.223.21 for the radish server/droplet
port = 5433 #my local port
database = 'membership_ard'
def get_connection():
	return sqlalchemy.create_engine(
		url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
			user, password, host, port, database
		)
	)

if __name__ == '__main__':

	try:
	
		# GET THE CONNECTION OBJECT (ENGINE) FOR THE DATABASE
		engine = get_connection()
		print(
			f"Connection to the {host} for user {user} created successfully.")
	except Exception as ex:
		print("Connection could not be made due to the following error: \n", ex)

Connection to the 127.0.0.1 for user lcalleja2 created successfully.


In [29]:
#test the above connection by submitting a query to the server

from sqlalchemy import text

query = '''select * from consolidated_mem_type limit 10'''
with engine.connect() as conn:
    df3 = pd.read_sql(text(query), con = conn)


In [ ]:
# DEFINE THE DATABASE CREDENTIALS
# LOCAL DB VERSION   LOCAL DB VERSION   LOCAL DB VERSION   LOCAL DB VERSION   LOCAL DB VERSION
credentials = Credentials()

cred_dict = credentials.retrieve_credentials()

user = cred_dict['user'] 
password = cred_dict['pass'] 
host = cred_dict['host'] 
port = cred_dict['port'] 
database = cred_dict['database'] 

def get_connection():
	return sqlalchemy.create_engine(
		url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
			user, password, host, port, database
		)
	)

if __name__ == '__main__':

	try:
	
		# GET THE CONNECTION OBJECT (ENGINE) FOR THE DATABASE
		engine = get_connection()
		print(
			f"Connection to the {host} for user {user} created successfully.")
	except Exception as ex:
		print("Connection could not be made due to the following error: \n", ex)


Connection to the 172.17.0.2 for user root created successfully.


### *Must delete the tables in mysql db or rename the tables before writing

In [31]:
#two DF names: type_df and cum_df_status
try:
    frame = cum_df_status.to_sql('mem_status_120425_ts', con=engine, if_exists='replace', index=False)
    #drop latest_trial as field is redundant
    #frame = type_df.drop('latest_trial', axis =1).to_sql('mem_type_120425_ts', con=engine, if_exists='replace', index=False)
except ValueError as vx:
    print(vx)
except Exception as ex:   
    print(ex)
else:
    print("Table %s created successfully."%'mem_status');   
finally:
    engine.dispose()

Table mem_status created successfully.


## End of DB injection code

### develop the program to join status activity to prevailing member type
- if trial related, the actvity will be assigned to one of: the trial with an active date range that encompasses the activity OR the historical trial that occurred prior to the trial having a start_dt AFTER the activity date of the status 
- once a person becomes a member, they will typically stay in that category, and it's up to me to toggle them active/leave/cancelled depending on status updates
- ultimately I care about: trial conversions, trial expirations/cancellations, 

Attempt to filter for trial members that do not upgrade and observe the permutations of subjects to learn which text is exclusive to trial memberships, so I can properly associate them/dismiss them when building the user journey

In [ ]:
cum_df_type.groupby('email').filter(lambda x: any([re.search('trial',i) for i in x['type']])).sort_values(['email','start_dt'])

In [ ]:
#output.groupby('email').filter(lambda x: len(x) == 2)
#trials = output.groupby('email').filter(lambda x: any(x['membership_type'] == 'trial'))
#output.groupby('email').filter(lambda x: any(x['membership_type'] == 'trial'))
#print(trials.shape)
#this version filters the groupby object by TWO conditions
output2 = cum_df_type.groupby('email').filter(lambda x: any([re.search('trial',i) for i in x['type']])).sort_values(['email','start_dt'])
print(output2.shape)
#inspect output of the program
output2.to_csv('./trial_type_sorted.csv', index = False)
cum_df_status.loc[cum_df_status['email'].isin(output2['email'])].sort_values(['email','start_dt']).to_csv('./trial_status_sorted.csv',index = False)

In [ ]:
#TODO print Output2 and arrive at the expected shape of the multi conditional (above)
#output2
size_one = output.groupby('email').filter(lambda x: len(x) == 1)
activityReport.loc[activityReport['email_grouping'].isin(size_one.loc[size_one['membership_type'].str.contains('trial'),'email']),].head()

In [ ]:

#enables reviewing all unique subjects associated to trials (excluding all other member types)
trials_only = activityReport.loc[activityReport['email_grouping'].isin(size_one.loc[size_one['membership_type'].str.contains('trial'),'email']),['email_grouping','Subject_act','Activity_Date_DT_act']].sort_values(['email_grouping','Activity_Date_DT_act'])
trials_only.to_csv('./trialsOnly.csv', index = False)

#consider only unique values of Subject_act
activityReport.loc[activityReport['email_grouping'].isin(size_one.loc[size_one['membership_type'].str.contains('trial'),'email']),['Subject_act']]['Subject_act'].unique()


In [ ]:
#view output of above
activityReport.loc[activityReport['email_grouping'].isin(output2['email']),].to_csv('./trialMembersFullOutput.csv', index = False)

In [ ]:
# in an attempt to identify status changes that are artifacts of an initial trial, select for all status changes after there is a type change (conversion of trial to other membership type)
# need to remove entries with Status cahnges: "Active to New", "New to Active", "Pending to New"
mult_groups = output.groupby(['email']).filter(lambda x: len(x) >1)
activityReport.loc[(activityReport['email_grouping'].isin(mult_groups['email'])) & (activityReport['email_grouping'].isin(output.loc[output['membership_type'].str.contains('trial'),'email'])),['email_grouping','Activity_Date_DT_act','Subject_act','Activity_Type_act']].sort_values(['email_grouping','Activity_Date_DT_act']).to_csv('./multi_w_trial.csv', index = False)

In [ ]:
pd.Series(activityReport.loc[activityReport['email_grouping'].isin(output2['email']),'Subject_act'].unique())[pd.Series(activityReport.loc[activityReport['email_grouping'].isin(output2['email']),'Subject_act'].unique()).str.contains('Status changed')]

### TODO: start going through event and type use cases, ex. Active to Expired for Trial (when within a two or 6 month period from Trial start)

In [ ]:
#best practice is always to sort this dataframe first
class_df = pd.DataFrame.from_dict(s.dict_loop('status'), orient = 'index', columns = ['start_date']).reset_index()
print(class_df)

In [ ]:
#best practice is always to sort this dataframe first
class_df = pd.DataFrame.from_dict(st.dict_loop('class'), orient = 'index', columns = ['start_date']).reset_index()
print(class_df)